In [1]:
import sys
sys.path.append('../utils/')
sys.path.append('../paviaUTools/')

import matplotlib.pyplot as plt
from datasetLoader import datasetLoader
import os
import numpy as np
from whole_pipeline import whole_pipeline_all, whole_pipeline_divided, whole_pipeline_divided_parallel
import torch
from plots import *
from weights_anal import *

import gc
torch.cuda.empty_cache()
gc.collect()

%matplotlib inline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
parent_dir = os.path.join(os.getcwd(),"..")
csv_path = os.path.join(parent_dir, 'datasets', 'paviaU.csv')
gt_path = os.path.join(parent_dir, 'datasets', 'paviaU_gt.csv')
# csv_path = os.path.join(parent_dir, 'datasets', 'pavia.csv')
# gt_path = os.path.join(parent_dir, 'datasets', 'pavia_gt.csv')

dsl = datasetLoader(csv_path, gt_path)

df = dsl.read_dataset(gt=False)
X = np.array(df)
X = X.reshape((610,340, 103))
# X = X.reshape((1096, 715, 102))

df = dsl.read_dataset(gt=True)
y = np.array(df)

factor = 11
rows_factor = factor
cols_factor = factor

X = torch.from_numpy(X)
y = torch.from_numpy(y)

X = X.to(device)
y = y.to(device)

In [3]:
import gc
from MetaLearner import HDDOnBands

avg_acc_train = 0.0
avg_acc_test = 0.0

NUM=10

for _ in range(NUM):
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_all(X,y, rows_factor, cols_factor, is_normalize_each_band=True, method_label_patch='most_common', is_random=False)
    avg_acc_train += train_acc/NUM
    avg_acc_test += test_acc/NUM

    print("train_acc: ", train_acc)
    print("test_acc: ", test_acc)

print("avg_acc_train: ", avg_acc_train)
print("avg_acc_test: ", avg_acc_test)



XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  5.640012979507446
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  34161
total classified:  8590
Train Accuracy:  0.9750300049764351
Test Accuracy:  0.7840512223515715
train_acc:  0.9750300049764351
test_acc:  0.7840512223515715
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  5.665869474411011
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  34161
total classified:  8590
Train Accuracy:  0.9750300049764351
Test Accuracy:  0.7840512223515715
train_acc:  0.9750300049764351
test_acc:  0.7840512223515715
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  5.380541086196899
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  34161
total classified:  8590
Train Accuracy:  0.9750300049764351
Test Accuracy:  0.7840512223515715
train_acc:  0.9750300049764351
test_acc:  0.7840512223515715
XXXXXXX IN METHOD XXXXXXXXX
WHOLE METHOD TIME:  5.196009397506714
XXXXXXX IN CLASSIFICATION XXXXXXXXX
total classified:  34161
total classified:  8590


KeyboardInterrupt: 